In [1]:
import torch
import numpy as np
from utils.generate_csbm import *
from utils.train import *
from utils.utils import *
from models.models import *
from models.setup import *
import networkx as nx
import pickle

In [55]:
def exp_csbm(ith, method, folder_path, csbm, n_train, n_val, 
             num_communication=100, batch_size=1,
             learning_rate=0.01, I=10, gradient=True, noise=False,
             Print=False, print_time=1,
             resume=False,
             bias=False,
             save=True,
             improved=False):
    

    init_mlp = MLP(csbm.Xs[0].shape[1], 64, 2, bias)
        
    if (gradient == False):
        grad = "no_grad/"
    elif (gradient == True and noise==False):
        grad = "biased_grad/"
    elif (gradient == True and noise==True):
        grad = "noisy_grad/"
      
    A_tilde = calculate_Atilde(csbm.A, K=10, alpha=0.9)
    server = set_up_GC(csbm.Xs, csbm.ys, init_mlp, A_tilde, n_train=n_train, n_val=n_val, improved=improved)
   
        
    tl, ta, vl, va = train_GC(server, num_communication, batch_size, learning_rate, I,
                              gradient, noise, 
                              Print, print_time)
    print (server.eval_test()[1])
    if save:    
        np.save(folder_path + method + grad + "I" + str(I) + "/tl_" + str(ith), tl)
        np.save(folder_path + method + grad + "I" + str(I) + "/ta_" + str(ith), ta)
        np.save(folder_path + method + grad + "I" + str(I) + "/vl_" + str(ith), vl)
        np.save(folder_path + method + grad + "I" + str(I) + "/va_" + str(ith), va)

        PATH = folder_path + method + grad + "I" + str(I) + "/model_" + str(ith)
        torch.save({
                'best_model_state_dict': server.best_cmodel.state_dict(),
                'learning_rate': learning_rate,
                'test_acc': server.eval_test()[1],
                'model_state_dict': server.cmodel.state_dict(),
                'best_valloss': server.best_valloss,
                'best_valacc': server.best_valacc,
                }, PATH)

In [66]:
for ith in range(20):
    for I in [10, 20, 50]:
        file_to_open= open("experiments/GC2/data/"+"csbm_"+str(ith)+".pickle", "rb")
        csbm = pickle.load(file_to_open)
        file_to_open.close()
        nc = int(2500/I)
        exp_csbm(ith=ith, method="GFLAPPNP/", folder_path="experiments/GC2/", csbm=csbm, n_train=10, n_val=10, 
                 num_communication=nc, batch_size=5,
                 learning_rate=0.2, I=I, gradient=False, noise=False,
                 Print=False, print_time=10,
                 resume=False,
                 bias=False,
                 save=True)
        print (ith, I)

0.7002000000000002
0 10
0.6994000000000001
0 20
0.7048000000000001
0 50
0.6603999999999999
1 10
0.6681999999999999
1 20
0.6576
1 50
0.6775999999999999
2 10
0.6829999999999999
2 20
0.6769999999999999
2 50
0.6852000000000003
3 10
0.6836000000000001
3 20
0.6833999999999999
3 50
0.7021999999999998
4 10
0.6997999999999999
4 20
0.7053999999999999
4 50
0.6984000000000004
5 10
0.6973999999999999
5 20
0.6966
5 50
0.6978
6 10
0.6873999999999999
6 20
0.6910000000000001
6 50
0.7019999999999996
7 10
0.6939999999999997
7 20
0.6983999999999999
7 50
0.6833999999999998
8 10
0.6809999999999999
8 20
0.679
8 50
0.6810000000000003
9 10
0.6828
9 20
0.6878
9 50
0.6906
10 10
0.6893999999999998
10 20
0.6888000000000001
10 50
0.6878000000000002
11 10
0.6839999999999999
11 20
0.6904000000000002
11 50
0.6987999999999996
12 10
0.6948000000000002
12 20
0.6987999999999999
12 50
0.6892
13 10
0.6928
13 20
0.6932
13 50
0.6990000000000001
14 10
0.6940000000000001
14 20
0.6989999999999997
14 50
0.6884000000000001
15 10
0

In [67]:
for ith in range(20):
    for I in [10]:
        file_to_open= open("experiments/GC2/data/"+"csbm_"+str(ith)+".pickle", "rb")
        csbm = pickle.load(file_to_open)
        file_to_open.close()
        nc = int(2500/I)
        exp_csbm(ith=ith, method="GFLAPPNP/", folder_path="experiments/GC2/", csbm=csbm, n_train=10, n_val=10, 
                 num_communication=nc, batch_size=5,
                 learning_rate=0.2, I=I, gradient=True, noise=True,
                 Print=False, print_time=10,
                 resume=False,
                 bias=False,
                 save=True)
        print (ith, I)

0.6910000000000003
0 10
0.6733999999999998
1 10
0.6854
2 10
0.6871999999999998
3 10
0.6995999999999998
4 10
0.6978000000000002
5 10
0.6916
6 10
0.6998000000000002
7 10
0.669
8 10
0.6716
9 10
0.6892
10 10
0.6833999999999999
11 10
0.6931999999999999
12 10
0.6780000000000002
13 10
0.6907999999999997
14 10
0.6851999999999999
15 10
0.7004000000000002
16 10
0.6731999999999999
17 10
0.6944
18 10
0.6848000000000002
19 10


In [72]:
for ith in range(17,20):
    for I in [20]:
        file_to_open= open("experiments/GC2/data/"+"csbm_"+str(ith)+".pickle", "rb")
        csbm = pickle.load(file_to_open)
        file_to_open.close()
        nc = int(2500/I)
        exp_csbm(ith=ith, method="GFLAPPNP/", folder_path="experiments/GC2/", csbm=csbm, n_train=10, n_val=10, 
                 num_communication=nc, batch_size=5,
                 learning_rate=0.15, I=I, gradient=True, noise=True,
                 Print=False, print_time=10,
                 resume=False,
                 bias=False,
                 save=True)
        print (ith, I)

0.6887999999999999
17 20
0.6933999999999999
18 20
0.6756
19 20


In [52]:
for ith in range(1,20):
    for I in [10, 20, 50]:
        file_to_open= open("experiments/GC2/data/"+"csbm_"+str(ith)+".pickle", "rb")
        csbm = pickle.load(file_to_open)
        file_to_open.close()
        nc = int(2000/I)
        exp_csbm(ith=ith, method="GFLAPPNP/", folder_path="experiments/GC2/", csbm=csbm, n_train=10, n_val=10, 
                 num_communication=nc, batch_size=5,
                 learning_rate=0.2, I=I, gradient=True, noise=False,
                 Print=False, print_time=10,
                 resume=False,
                 bias=False,
                 save=True)
        print (ith, I)

1 10
1 20
1 50
2 10
2 20
2 50
3 10
3 20
3 50
4 10
4 20
4 50
5 10
5 20
5 50
6 10
6 20
6 50
7 10
7 20
7 50
8 10
8 20
8 50
9 10
9 20
9 50
10 10
10 20
10 50
11 10
11 20
11 50
12 10
12 20
12 50
13 10
13 20
13 50
14 10
14 20
14 50
15 10
15 20
15 50
16 10
16 20
16 50
17 10
17 20
17 50
18 10
18 20
18 50
19 10
19 20
19 50


In [76]:
testaccs_mlps = []
for ith in range(20):
    file_to_open= open("experiments/GC2/data/"+"csbm_"+str(ith)+".pickle", "rb")
    csbm = pickle.load(file_to_open)
    file_to_open.close()
    init_mlp = MLP(csbm.Xs[0].shape[1], 64, 2, bias=False)
    server_MLPs = set_up_MLP_GC(csbm.Xs, csbm.ys, init_mlp, n_train=10, n_val=10)
    server_MLPs.train_nodes(batch_size=5, learning_rate=0.05, num_epoch=200, Print=False, print_time=50)
    testaccs_mlps.append(server_MLPs.eval_test()[1])
    print (ith)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [82]:
testaccs_fedmlp = {10: [], 20: [], 50: []}
for ith in range(20):
    for I in [10, 20, 50]:
        nc = int(1000/I)
        file_to_open= open("experiments/GC2/data/"+"csbm_"+str(ith)+".pickle", "rb")
        csbm = pickle.load(file_to_open)
        file_to_open.close()
        init_mlp = MLP(csbm.Xs[0].shape[1], 64, 2, bias=False)
        server_FedMLP = set_up_FedMLP_GC(csbm.Xs, csbm.ys, init_mlp, n_train=10, n_val=10)
        train_FedMLP(server_FedMLP, num_communication=nc, 
                 batch_size=5, learning_rate=0.08, I=I, 
                 Print=False, print_time=10)
        testaccs_fedmlp[I].append(server_FedMLP.eval_test()[1])
        print (ith, np.mean(testaccs_fedmlp[I]))

0 0.6012000000000001
0 0.6122000000000001
0 0.6085999999999998
1 0.6019000000000001
1 0.6096999999999999
1 0.6056999999999999
2 0.6030666666666668
2 0.6079333333333333
2 0.6065333333333333
3 0.60195
3 0.6071
3 0.6068
4 0.60544
4 0.60692
4 0.6062000000000001
5 0.6087333333333332
5 0.6096333333333334
5 0.6085666666666667
6 0.6078
6 0.6085714285714287
6 0.6068
7 0.610725
7 0.6105
7 0.60995
8 0.6084
8 0.6080444444444445
8 0.6092888888888889
9 0.60744
9 0.6065
9 0.60744
10 0.6068181818181818
10 0.6057636363636364
10 0.6058545454545454
11 0.6075499999999999
11 0.60585
11 0.6063333333333333
12 0.6082461538461538
12 0.6066923076923078
12 0.6067846153846154
13 0.6094142857142858
13 0.6074857142857143
13 0.6074571428571428
14 0.61012
14 0.6080133333333333
14 0.6083199999999999
15 0.60995
15 0.608075
15 0.6084875000000001
16 0.6109176470588236
16 0.6091294117647059
16 0.6094823529411766
17 0.6112
17 0.6091777777777777
17 0.6094777777777779
18 0.6108736842105263
18 0.6095578947368421
18 0.60977894

In [83]:
np.save("testaccs_mlps", testaccs_mlps)

In [85]:
np.save("testaccs_fedmlp_I10", testaccs_fedmlp[10])
np.save("testaccs_fedmlp_I20", testaccs_fedmlp[20])
np.save("testaccs_fedmlp_I50", testaccs_fedmlp[50])